In [ ]:
# We launch the libraries
library(jsonlite)
library(rlist)
library(data.table)
library(XML)
library(methods)

In [6]:
# We select the xmls with the datas we need
setwd('/home/jovyan/xml/')


In [ ]:
suppressWarnings({
options(stringsAsFactors = FALSE)
xmlproducts3=list.files(pattern='product3')
# We put all the xml products3 from orphadata in data frames
indice_list=1
xmldoc=list()
products3=list()
node_names=list()
names=list()
for (i in 1:length(xmlproducts3)) 
{
  # products3
  xmldoc[[indice_list]]=xmlParse(xmlproducts3[i]) # we parse the products3
  products3[[indice_list]]=xmlToDataFrame(nodes = getNodeSet(xmldoc[[i]], "//Disorder")) # we put products3 into a list
  node_names[[indice_list]]=xmlToDataFrame(nodes = getNodeSet(xmldoc[[i]], "//Classification"))
  names[[indice_list]]=node_names[[indice_list]]$Name
  indice_list=indice_list+1
}
})

In [8]:
# We paste the name of the xml, which represents a medical speciality
indice_list=1
product3=list()
for (i in 1:length(products3))
{
  product3[[indice_list]]=cbind.data.frame(products3[[i]],names[[i]])
  indice_list=indice_list+1
}
product3=rbindlist(product3,use.names = TRUE)
names(product3)=c('OrphaCode','ExpertLink','Name','DisorderType','MedicalSpeciality')

In [9]:
# We only keep Disorders
indice_list=1
products3_disorders=list()
for (i in 1:nrow(product3))
    if (product3$DisorderType[i]=='Particular clinical situation in a disease or syndrome' |
        product3$DisorderType[i]=='Biological anomaly' |
        product3$DisorderType[i]=='Malformation syndrome' |
        product3$DisorderType[i]=='Morphological anomaly' |
        product3$DisorderType[i]=='Disease' |
        product3$DisorderType[i]=='Clinical syndrome')
    {
      products3_disorders[[indice_list]]=product3[i, ]
      indice_list=indice_list+1
    }
products3_disorders=rbindlist(products3_disorders,use.names = TRUE)

#We remove the duplicates
products3_disorders=products3_disorders[!duplicated(products3_disorders)]

In [10]:
# We count the number of OrphaCodes in each xml (in each medical speciality)
medical_speciality=as.data.frame(table(products3_disorders$MedicalSpeciality))
names(medical_speciality)=c('MedicalSpeciality','Number')
medical_speciality

MedicalSpeciality,Number
<fct>,<int>
Orphanet classification of rare abdominal surgical diseases,160
Orphanet classification of rare allergic disease,14
Orphanet classification of rare bone diseases,804
Orphanet classification of rare cardiac diseases,149
Orphanet classification of rare cardiac malformations,172
Orphanet classification of rare circulatory system diseases,209
Orphanet classification of rare developmental anomalies during embryogenesis,2696
Orphanet classification of rare disorders due to toxic effects,55
Orphanet classification of rare endocrine diseases,446
